In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
cd /content/drive/MyDrive/PSR

In [ ]:
!pip install scikit-plot

In [ ]:
#LOAD DEPENDENCIES
import os
import time
import pickle
import logging
import scikitplot 
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

#IMPORT LOSS, OPTIMIZER, CALLBACK AND LAYERS
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy, KLDivergence
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.layers import Input, Dense, Dropout, GlobalAveragePooling2D

#IMPORT MODEL APIs
from tensorflow.keras.models import Model, load_model, save_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2 as selected_model
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input as student_preprocess

import itertools
import matplotlib.pyplot as plt
import matplotlib.patheffects as path_effects
from sklearn.metrics import accuracy_score, mean_squared_error, mean_squared_log_error, classification_report, confusion_matrix, roc_curve, auc

#PREVENT ERROR UNCESSARY MESSAGES
tf.get_logger().setLevel(logging.ERROR)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

print("LIBRARIES LOADED")

In [ ]:
#Tweakable parameters
MODEL_KIND = "proposed_model"
MODEL_NAME = "CommonBase"

#Models paths
MODEL_PATH = "models/" + MODEL_KIND  + '/' + MODEL_NAME

#Figures paths
FIG_PATH = 'figures/' + MODEL_KIND + "/" + MODEL_NAME

#Data paths
MAIN_DATA_DIR = "ds/"
TRAIN_DATA_DIR = MAIN_DATA_DIR + "train/"
TEST_DATA_DIR = MAIN_DATA_DIR + "test/"
VALIDATION_DATA_DIR = MAIN_DATA_DIR + "val/"

print("ALL REQUERED PATHS SET")

In [ ]:
#Load model Function
def load_m(directory, model_name, custom_objects=None):
    if not os.path.exists(directory):
        print("Model File Does Not Exist!!")
        return 
    model = load_model(directory + "/" + model_name + ".h5", custom_objects=custom_objects)
    model.summary()
    return model

#Load History Function
def load_h(directory, history_name):
    if not os.path.exists(directory):
        print("History File Does Not Exist!!")
        return 
    with open(directory + '/' + history_name + '.history', 'rb') as file:
        his = pickle.load(file)
    return his

def save_fig(directory, fig_name):
    if not os.path.exists(directory):
        os.makedirs(directory)
    plt.savefig(directory + '/' + fig_name + '.tiff', bbox_inches='tight', dpi=600, format='tiff')

In [ ]:
BATCH_SIZE = 4
img_rows, img_cols = (224, 224)

#LOAD VAL DATA
val_datagen = ImageDataGenerator(preprocessing_function=student_preprocess)

if not os.path.exists(TRAIN_DATA_DIR):
    print("VALIDATION DATA DOES NOT EXITS!")
else:
    print("LOAD VALIDATION SAMPLES...")
    validation_generator = val_datagen.flow_from_directory(
                VALIDATION_DATA_DIR,
                target_size=(img_rows,img_cols),
                batch_size=BATCH_SIZE,
                class_mode='categorical',
                seed=42,
                shuffle=False)

    #CHECK  THE NUMBER OF SAMPLES
    nb_validation_samples = len(validation_generator.filenames)
    if nb_validation_samples == 0:
        print("NO DATA VALIDATION FOUND IN VALIDATION FOLDER!")

#LOAD TEST DATA
test_datagen = ImageDataGenerator(preprocessing_function=student_preprocess)

if not os.path.exists(TEST_DATA_DIR):
    print("TEST DATA DOES NOT EXITS!")
else:
    print("LOAD TEST SAMPLES...")
    test_generator = test_datagen.flow_from_directory(
                TEST_DATA_DIR,
                target_size=(img_rows,img_cols),
                batch_size=BATCH_SIZE,
                class_mode='categorical',
                seed=42,
                shuffle=False)

#CHECK  THE NUMBER OF SAMPLES
nb_test_samples = len(test_generator.filenames)
if nb_test_samples == 0:
    print("NO DATA TEST FOUND IN TEST FOLDER!")

In [ ]:
#Load the trained model into model variable for consitency.
model = load_m(MODEL_PATH, MODEL_NAME)

In [ ]:
#Sanity check
model.evaluate(validation_generator, return_dict=True)

#Test the model
model.evaluate(test_generator, return_dict=True)

In [ ]:
# get predictions
y_pred = model.predict(validation_generator, 
                                 nb_validation_samples/BATCH_SIZE, workers=1, verbose=1)

y_pred_2 = model.predict(test_generator, 
                                 nb_test_samples/BATCH_SIZE, workers=1, verbose=1)

#actual targets
Y_test=validation_generator.classes
Y_test_2=test_generator.classes

In [ ]:
axisplot1 = scikitplot.metrics.plot_roc(Y_test, y_pred, cmap='tab10',classes_to_plot=[])
plt.title('BaseComMon_ROC_curve-VAL', fontsize=18, fontfamily='Tahoma')

plt.xlabel('Specificity', fontsize=18, fontfamily='Tahoma')
plt.ylabel('Sensitivity', fontsize=18, fontfamily='Tahoma')

plt.tick_params(length = 5, 
                width = 2, 
                right = True, 
                top = True, 
                labelsize = 12)

plt.rc('xtick', direction="in") 
plt.rc('ytick', direction="in") 
legend = plt.legend(loc="lower right", fontsize=11.5, labelspacing=0.5)
plt.setp(legend.texts, family = 'Tahoma')

plt.tight_layout()
save_fig(FIG_PATH, 'ROC_curve-VAL')
plt.show()

In [ ]:
axisplot2 = scikitplot.metrics.plot_roc(Y_test_2, y_pred_2, cmap='tab10',classes_to_plot=[])
plt.title('BaseComMon_ROC_curve-Test', fontsize=18, fontfamily='Tahoma')

plt.xlabel('Specificity', fontsize=18, fontfamily='Tahoma')
plt.ylabel('Sensitivity', fontsize=18, fontfamily='Tahoma')

plt.tick_params(length = 5, 
                width = 2, 
                right = True, 
                top = True, 
                labelsize = 12)
   
plt.rc('xtick', direction="in") 
plt.rc('ytick', direction="in") 
legend = plt.legend(loc="lower right", fontsize=11.5, labelspacing=0.5)
plt.setp(legend.texts, family = 'Tahoma')

plt.tight_layout()
save_fig(FIG_PATH, 'ROC_curve-TEST')
plt.show()

In [ ]:
axisplot3 = scikitplot.metrics.plot_precision_recall(Y_test, y_pred, cmap='tab10',classes_to_plot=[])
plt.title('BaseComMon_PR_curve-VAL', fontsize=18, fontfamily='Tahoma')

plt.xlabel('Recall', fontsize=18, fontfamily='Tahoma')
plt.ylabel('Precision', fontsize=18, fontfamily='Tahoma')

plt.tick_params(length = 5, 
                width = 2, 
                right = True, 
                top = True, 
                labelsize = 12)
   
plt.rc('xtick', direction="in") 
plt.rc('ytick', direction="in") 
legend = plt.legend(loc="lower right", fontsize=11.5, labelspacing=0.5)
plt.setp(legend.texts, family = 'Tahoma')

plt.tight_layout()
save_fig(FIG_PATH, 'PR_curve-VAL')
plt.show()

In [ ]:
axisplot4 = scikitplot.metrics.plot_precision_recall(Y_test_2, y_pred_2, cmap='tab10',classes_to_plot=[])
plt.title('BaseComMon_PR_curve-Test', fontsize=18, fontfamily='Tahoma')

plt.xlabel('Recall', fontsize=18, fontfamily='Tahoma')
plt.ylabel('Precision', fontsize=18, fontfamily='Tahoma')

plt.tick_params(length = 5, 
                width = 2, 
                right = True, 
                top = True, 
                labelsize = 12)
   
plt.rc('xtick', direction="in") 
plt.rc('ytick', direction="in") 
legend = plt.legend(loc="lower right", fontsize=11.5, labelspacing=0.5)
plt.setp(legend.texts, family = 'Tahoma')

plt.tight_layout()
save_fig(FIG_PATH, 'PR_curve-TEST')
plt.show()

In [ ]:
#Convert to excel data
import pandas as pd

def export_excel(plot, path, name):
    if not os.path.exists(path):
        os.makedirs(path)

    path = path + '/' + name
    lines = plot.axes.get_lines()
    lines = list(lines)
    all_data = []

    for line in lines:
        label = line.get_label()
        excel_data = {}
        excel_data[label + '-x'] = {}
        excel_data[label + '-y'] = {}

        x = line.get_xdata()
        y = line.get_ydata()

        for i in range(len(x)):
            excel_data[label + '-x'][i] = x[i]

        for i in range(len(y)):
            excel_data[label + '-y'][i] = y[i]

        df = pd.DataFrame(excel_data)  
        all_data.append(df)

    all_df = pd.concat(all_data, axis=1)
    all_df.to_excel(path)
    print("Plot Exported to ", path)

In [ ]:
export_excel(axisplot1, 'raw_excel_data/ComMonBase','ROC_curve-VAL.xlsx')
export_excel(axisplot2, 'raw_excel_data/ComMonBase','ROC_curve-TEST.xlsx')
export_excel(axisplot3, 'raw_excel_data/ComMonBase','PR_curve-VAL.xlsx')
export_excel(axisplot4, 'raw_excel_data/ComMonBase','PR_curve-TEST.xlsx')

In [ ]:
#Tweakable parameters
MODEL_KIND = "proposed_model"
MODEL_NAME = "MiniMobileNetV2"

#Models paths
MODEL_PATH = "models/" + MODEL_KIND  + '/' + MODEL_NAME

#Figures paths
FIG_PATH = 'figures/' + MODEL_KIND + "/" + MODEL_NAME

In [ ]:
#Load the trained model into model variable for consitency.
model = load_m(MODEL_PATH, MODEL_NAME)

In [ ]:
#Sanity check
model.compile(loss=CategoricalCrossentropy(), optimizer='adam', metrics=['accuracy'])
model.evaluate(validation_generator, return_dict=True)

#Test the model
model.evaluate(test_generator, return_dict=True)

In [ ]:
# get predictions
y_pred = model.predict(validation_generator, 
                                 nb_validation_samples/BATCH_SIZE, workers=1, verbose=1)

y_pred_2 = model.predict(test_generator, 
                                 nb_test_samples/BATCH_SIZE, workers=1, verbose=1)

#actual targets
Y_test=validation_generator.classes
Y_test_2=test_generator.classes

In [ ]:
axisplot1 = scikitplot.metrics.plot_roc(Y_test, y_pred, cmap='tab10',classes_to_plot=[])
plt.title('KDComMon_ROC_curve-VAL', fontsize=18, fontfamily='Tahoma')

plt.xlabel('Specificity', fontsize=18, fontfamily='Tahoma')
plt.ylabel('Sensitivity', fontsize=18, fontfamily='Tahoma')

plt.tick_params(length = 5, 
                width = 2, 
                right = True, 
                top = True, 
                labelsize = 12)

plt.rc('xtick', direction="in") 
plt.rc('ytick', direction="in") 
legend = plt.legend(loc="lower right", fontsize=11.5, labelspacing=0.5)
plt.setp(legend.texts, family = 'Tahoma')

plt.tight_layout()
save_fig(FIG_PATH, 'ROC_curve-VAL')
plt.show()

In [ ]:
axisplot2 = scikitplot.metrics.plot_roc(Y_test_2, y_pred_2, cmap='tab10',classes_to_plot=[])
plt.title('KDComMon_ROC_curve-Test', fontsize=18, fontfamily='Tahoma')

plt.xlabel('Specificity', fontsize=18, fontfamily='Tahoma')
plt.ylabel('Sensitivity', fontsize=18, fontfamily='Tahoma')

plt.tick_params(length = 5, 
                width = 2, 
                right = True, 
                top = True, 
                labelsize = 12)
   
plt.rc('xtick', direction="in") 
plt.rc('ytick', direction="in") 
legend = plt.legend(loc="lower right", fontsize=11.5, labelspacing=0.5)
plt.setp(legend.texts, family = 'Tahoma')

plt.tight_layout()
save_fig(FIG_PATH, 'ROC_curve-TEST')
plt.show()

In [ ]:
axisplot3 = scikitplot.metrics.plot_precision_recall(Y_test, y_pred, cmap='tab10',classes_to_plot=[])
plt.title('KDComMon_PR_curve-VAL', fontsize=18, fontfamily='Tahoma')

plt.xlabel('Recall', fontsize=18, fontfamily='Tahoma')
plt.ylabel('Precision', fontsize=18, fontfamily='Tahoma')

plt.tick_params(length = 5, 
                width = 2, 
                right = True, 
                top = True, 
                labelsize = 12)
   
plt.rc('xtick', direction="in") 
plt.rc('ytick', direction="in") 
legend = plt.legend(loc="lower right", fontsize=11.5, labelspacing=0.5)
plt.setp(legend.texts, family = 'Tahoma')

plt.tight_layout()
save_fig(FIG_PATH, 'PR_curve-VAL')
plt.show()

In [ ]:
axisplot4 = scikitplot.metrics.plot_precision_recall(Y_test_2, y_pred_2, cmap='tab10',classes_to_plot=[])
plt.title('KDComMon_PR_curve-Test', fontsize=18, fontfamily='Tahoma')

plt.xlabel('Recall', fontsize=18, fontfamily='Tahoma')
plt.ylabel('Precision', fontsize=18, fontfamily='Tahoma')

plt.tick_params(length = 5, 
                width = 2, 
                right = True, 
                top = True, 
                labelsize = 12)
   
plt.rc('xtick', direction="in") 
plt.rc('ytick', direction="in") 
legend = plt.legend(loc="lower right", fontsize=11.5, labelspacing=0.5)
plt.setp(legend.texts, family = 'Tahoma')

plt.tight_layout()
save_fig(FIG_PATH, 'PR_curve-TEST')
plt.show()

In [ ]:
export_excel(axisplot1, 'raw_excel_data/KDComMon','ROC_curve-VAL.xlsx')
export_excel(axisplot2, 'raw_excel_data/KDComMon','ROC_curve-TEST.xlsx')
export_excel(axisplot3, 'raw_excel_data/KDComMon','PR_curve-VAL.xlsx')
export_excel(axisplot4, 'raw_excel_data/KDComMon','PR_curve-TEST.xlsx')